In [ ]:
# !pip install econml

In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

# BHP: ForestRiesz

## Library Imports

In [ ]:
import os
import glob
from joblib import dump, load
import pandas as pd
import scipy
import scipy.stats
import scipy.special
import numpy as np
import math
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_predict
from utils.forestriesz import poly_feature_fns
from utils.RF_avgmom_sim import sim_fun

## Moment Definition

In [ ]:
def moment_fn(X, test_fn): # Returns the moment for the average small difference
    epsilon = 0.01
    t1 = np.hstack([X[:, [0]] + epsilon, X[:, 1:]])
    t0 = np.hstack([X[:, [0]] - epsilon, X[:, 1:]])
    return (test_fn(t1) - test_fn(t0)) / (2*epsilon)

## Settings for RF Estimators

In [ ]:
# RFRiesz Settings
ForestRiesz_opt = {
    'reg_feature_fns': poly_feature_fns(1),
    'riesz_feature_fns': poly_feature_fns(3),
    'moment_fn': moment_fn,
    'l2': 1e-3,
    'criterion': 'mse', 
    'n_estimators': 100,
    'min_samples_leaf': 50,
    'min_var_fraction_leaf': 0.1, 
    'min_var_leaf_on_val': True,
    'min_impurity_decrease': 0.001, 
    'max_samples': .65, 
    'max_depth': None,
    'warm_start': False,
    'inference': False,
    'subforest_size': 1,
    'honest': True,
    'verbose': 0,
    'n_jobs': -1,
    'random_state': 123
}

# RFreg Settings
RFreg_opt = {
    'reg_feature_fns': poly_feature_fns(1),
    'l2': 1e-3,
    'criterion': 'mse', 
    'n_estimators': 100,
    'min_samples_leaf': 50,
    'min_var_fraction_leaf': 0.1, 
    'min_var_leaf_on_val': True,
    'min_impurity_decrease': 0.001, 
    'max_samples': .65, 
    'max_depth': None,
    'warm_start': False,
    'inference': False,
    'subforest_size': 1,
    'honest': True,
    'verbose': 0,
    'n_jobs': -1,
    'random_state': 123
}

# RFrr Settings
RFrr_opt = {
    'riesz_feature_fns': poly_feature_fns(3),
    'moment_fn': moment_fn, 
    'l2': 1e-3,
    'criterion': 'mse', 
    'n_estimators': 100,
    'min_samples_leaf': 50,
    'min_var_fraction_leaf': 0.1, 
    'min_var_leaf_on_val': True,
    'min_impurity_decrease': 0.001, 
    'max_samples': .65, 
    'max_depth': None,
    'warm_start': False,
    'inference': False,
    'subforest_size': 1,
    'honest': True,
    'verbose': 0,
    'n_jobs': -1,
    'random_state': 123
}

## Read Data

In [ ]:
df = pd.read_csv('./data/BHP/data_BHP2.csv')
df = df[df["log_p"] > math.log(1.2)]
df = df[df["log_y"] > math.log(15000)]
Xdf = df.iloc[:,1:]
X_nostatedum = Xdf.drop(["distance_oil1000", "share"], axis=1).values
columns = Xdf.columns
state_dum = pd.get_dummies(Xdf['state_fips'], prefix="state")
Xdf = pd.concat([Xdf, state_dum], axis = 1)
Xdf = Xdf.drop(["distance_oil1000", "state_fips", "share"], axis=1)
W = Xdf.drop(["log_p"], axis=1).values
T = Xdf['log_p'].values

## Generate Semi-Synthetic Treatment

In [ ]:
# Conditional Mean
mu_T = RandomForestRegressor(n_estimators = 100, min_samples_leaf = 50, random_state = 123)
mu_T.fit(W, T)

# Conditional Variance
sigma2_T = RandomForestRegressor(n_estimators = 100, min_samples_leaf = 50, max_depth = 5, random_state = 123)
e_T = T - cross_val_predict(mu_T, W, T)
sigma2_T.fit(W, e_T ** 2)

In [ ]:
def gen_T(W): # T ~ N(\mu(W), \sigma^2(W))
    n = W.shape[0]
    return (mu_T.predict(W) + np.sqrt(sigma2_T.predict(W)) * np.random.normal(size=(n,))).reshape(-1,1)

def true_rr(X):
    return (X[:, 0] - mu_T.predict(X[:, 1:]))/(sigma2_T.predict(X[:, 1:]))

## Run Simulations

In [ ]:
for i in range(10):
    np.random.seed(i)

    b = np.random.uniform(-0.5, 0.5, size=(20, 1))
    c = np.random.uniform(-0.2, 0.2, size=(8, 1))

    def nonlin(X):
        return 1.5*scipy.special.expit(10 * X[:, 6]) + 1.5*scipy.special.expit(10 * X[:, 8])

    def true_f_simple(X):
        return -0.6 * X[:, 0]

    def true_f_simple_lin_conf(X):
        return true_f_simple(X) + np.matmul(X[:, 1:21], b).flatten()

    def true_f_simple_nonlin_conf(X):
        return true_f_simple_lin_conf(X) + nonlin(X)

    def true_f_compl(X):
        return -0.5 * (X[:, 1]**2/10 + .5) * X[:, 0]**3 / 3

    def true_f_compl_lin_conf(X):
        return -0.5 * (X[:, 1]**2/10 + np.matmul(X[:, 1:9], c).flatten() + .5) * X[:, 0]**3 / 3 + np.matmul(X[:, 1:21], b).flatten()

    def true_f_compl_nonlin_conf(X):
        return true_f_compl_lin_conf(X) + nonlin(X)

    for true_f in [true_f_simple, true_f_simple_lin_conf, true_f_simple_nonlin_conf,
                   true_f_compl, true_f_compl_lin_conf, true_f_compl_nonlin_conf]:
        print("Now trying " + true_f.__name__)

        def gen_y(X):
            n = X.shape[0]
            return true_f(X) + np.random.normal(0, np.sqrt(5.6 * np.var(true_f(X))), size = (n,))
        
        path = './results/BHP/ForestRiesz/' + true_f.__name__

        if not os.path.exists(path):
            os.makedirs(path)
            
        for xfit, mult in [(0, True), (0, False), (1, True), (1, False), (2, False)]:
            namedata = path + '/xfit_' + str(xfit) + '_mult_' + str(int(mult)) + '_seed_' + str(i) + '.joblib'
            nameplot = path + '/xfit_' + str(xfit) + '_mult_' + str(int(mult)) + '_seed_' + str(i) + '.pdf'
            sim_fun(W, moment_fn = moment_fn, true_reg = true_f, true_rr = true_rr, gen_y = gen_y, gen_T = gen_T, N_sim = 100, 
                    xfit = xfit, multitasking = mult, ForestRiesz_opt = ForestRiesz_opt, RFreg_opt = RFreg_opt, RFrr_opt = RFrr_opt,
                    seed = i, verbose = 0, plot = True, save = namedata, saveplot = nameplot)

## Summary Outputs

### LaTeX Table

In [ ]:
f_string = ["1. Simple $f$",
            "2. Simple $f$ with linear confound.",
            "3. Simple $f$ with linear and non-linear confound.",
            "4. Complex $f$",
            "5. Complex $f$ with linear confound.",
            "6. Complex $f$ with linear and non-linear confound."]

true_fs = ['true_f_simple', 'true_f_simple_lin_conf', 'true_f_simple_nonlin_conf',
           'true_f_compl', 'true_f_compl_lin_conf', 'true_f_compl_nonlin_conf']

methods = ['reg', 'ips', 'dr', 'tmle']
    
with open("./results/BHP/ForestRiesz/res_avg_der_RF.tex", "w") as f:
    f.write("\\begin{tabular}{*{16}{r}} \n" +
            "\\toprule \n" +
            "&&&& \\multicolumn{3}{c}{Direct} & \\multicolumn{3}{c}{IPS} & \\multicolumn{3}{c}{DR} & \\multicolumn{3}{c}{TMLE-DR} \\\\ \n" +
            "\\cmidrule(lr){5-7} \\cmidrule(lr){8-10} \\cmidrule(lr){11-13} \\cmidrule(lr){14-16} \n" +
            "x-fit & multit. & reg $R^2$ &  rr $R^2$ &  Bias &  RMSE &  Cov. &  Bias &  RMSE &  Cov. &  Bias &  RMSE &  Cov. &  Bias &  RMSE &  Cov. \\\\ \n" +
            "\\midrule \n")
    
    for f_i, true_f in enumerate(true_fs):
        path = './results/BHP/ForestRiesz/' + true_f
        f.write("\\addlinespace \n \\multicolumn{16}{l}{\\textbf{" + f_string[f_i] + "}} \\\\ \n")
        
        for xfit, mult in [(0, True), (0, False), (1, True), (1, False), (2, False)]:
            mult_str = "Yes" if mult else "No"
            
            f.write(" & ".join([str(xfit), mult_str]) + " & ")
            
            r2_reg, r2_rr = [], []
            res = {}
            for method in methods:
                res[method] = {'bias': [], 'rmse': [], 'cov': []}
            
            for i in range(10):
                namedata = path + '/xfit_' + str(xfit) + '_mult_' + str(int(mult)) + '_seed_' + str(i) + '.joblib'
                loaded = load(namedata)
                r2_reg = np.append(r2_reg, loaded[2])
                r2_rr = np.append(r2_rr, loaded[4])
                
                for method in methods:
                    res[method]['bias'].append(loaded[0][method]['bias'])
                    res[method]['rmse'].append(loaded[0][method]['rmse'])
                    res[method]['cov'].append(loaded[0][method]['cov'])
            
            f.write(" & ".join(["${:.3f}$".format(np.mean(x)) for x in [r2_reg, r2_rr]]) + " & ")
            f.write(" & ".join(["${:.3f}$".format(np.mean(res[method][x])) for method in methods
                                for x in ['bias', 'rmse', 'cov']]) + " \\\\ \n")

    f.write("\\bottomrule \n \\end{tabular}")

### RF Plug-in Benchmark

In [ ]:
f_string = ["1. Simple $f$",
            "2. Simple $f$ with linear confound.",
            "3. Simple $f$ with linear and non-linear confound.",
            "4. Complex $f$",
            "5. Complex $f$ with linear confound.",
            "6. Complex $f$ with linear and non-linear confound."]

true_fs = ['true_f_simple', 'true_f_simple_lin_conf', 'true_f_simple_nonlin_conf',
           'true_f_compl', 'true_f_compl_lin_conf', 'true_f_compl_nonlin_conf']
    
with open("./results/BHP/plugin.tex", "w") as f:
    f.write("\\begin{tabular}{*{5}{r}} \n" +
            "\\toprule \n" +
            "&& \\multicolumn{3}{c}{RF Plug-in} \\\\ \n" +
            "\\cmidrule(lr){3-5} \n" +
            "reg $R^2$ &  rr $R^2$ &  Bias &  RMSE &  Cov. \\\\ \n" +
            "\\midrule \n")
    
    for f_i, true_f in enumerate(true_fs):
        path = './results/BHP/ForestRiesz/' + true_f
        f.write("\\addlinespace \n \\multicolumn{5}{l}{\\textbf{" + f_string[f_i] + "}} \\\\ \n")
        
        r2_reg, r2_rr = [], []
        plugin = {'bias': [], 'rmse': [], 'cov': []}
            
        for i in range(10):
            namedata = path + '/xfit_0_mult_0_seed_' + str(i) + '.joblib'
            loaded = load(namedata)
            r2_reg = np.append(r2_reg, loaded[2])
            r2_rr = np.append(r2_rr, loaded[4])
            plugin['bias'].append(loaded[0]['plugin']['bias'])
            plugin['rmse'].append(loaded[0]['plugin']['rmse'])
            plugin['cov'].append(loaded[0]['plugin']['cov'])
            
        f.write(" & ".join(["${:.3f}$".format(np.mean(x)) for x in [r2_reg, r2_rr]]) + " & ")
        f.write(" & ".join(["${:.3f}$".format(np.mean(plugin[x]))
                            for x in ['bias', 'rmse', 'cov']]) + " \\\\ \n")

    f.write("\\bottomrule \n \\end{tabular}")

### Histograms over 10 Seeds

In [ ]:
methods = ['reg', 'ips', 'dr', 'tmle']

for true_f in true_fs: 
    
    path = './results/BHP/ForestRiesz/' + true_f
    
    for xfit, mult in [(0, True), (0, False), (1, True), (1, False), (2, False)]:            
            rmse_reg, r2_reg, rmse_rr, r2_rr, ipsbias, drbias, truth = [], [], [], [], [], [], []
            res = {}
            
            for method in methods:
                res[method] = {'point' : [], 'bias': [], 'rmse': [], 'cov': []}
            
            for i in range(10):
                namedata = path + '/xfit_' + str(xfit) + '_mult_' + str(int(mult)) + '_seed_' + str(i) + '.joblib'
                loaded = load(namedata)
                rmse_reg = np.append(rmse_reg, loaded[1])
                r2_reg = np.append(r2_reg, loaded[2])
                rmse_rr = np.append(rmse_rr, loaded[3])
                r2_rr = np.append(r2_rr, loaded[4])
                ipsbias = np.append(ipsbias, loaded[5])
                drbias = np.append(drbias, loaded[6])
                truth = np.append(truth, loaded[7])
                
                for method in methods:
                    res[method]['point'] = np.append(res[method]['point'], loaded[0][method]['point'])
                    res[method]['bias'].append(loaded[0][method]['bias'])
                    res[method]['rmse'].append(loaded[0][method]['rmse'])
                    res[method]['cov'].append(loaded[0][method]['cov'])
            
            nuisance_str = ("reg RMSE: {:.3f}, R2: {:.3f}, rr RMSE: {:.3f}, R2: {:.3f}\n"
                            "IPS orthogonality: {:.3f}, DR orthogonality: {:.3f}").format(np.mean(rmse_reg), np.mean(r2_reg),
                                                                                          np.mean(rmse_rr), np.mean(r2_rr),
                                                                                          np.mean(ipsbias), np.mean(drbias))
            method_strs = ["{}. Bias: {:.3f}, RMSE: {:.3f}, Coverage: {:.3f}".format(method, np.mean(d['bias']), np.mean(d['rmse']), np.mean(d['cov']))
                           for method, d in res.items()]
            plt.title("\n".join([nuisance_str] + method_strs))
            for method, d in res.items():
                plt.hist(np.array(d['point']), alpha=.5, label=method)
            plt.axvline(x = np.mean(truth), label='true', color='red')
            plt.legend()
            nameplot = path + '/xfit_' + str(xfit) + '_mult_' + str(int(mult)) + '_all.pdf'
            plt.savefig(nameplot, bbox_inches='tight')
            plt.show()